In [ ]:
import numpy as np
# !pip install opencv-contrib-python==3.4.2.17
import cv2
from sklearn.cluster import KMeans
from sklearn.model_selection import StratifiedKFold   
import keras
from keras.datasets import cifar10
from scipy.cluster.vq import *
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import sklearn.metrics as metrics

In [ ]:
!pip install opencv-contrib-python==4.4.0.44

     |████████████████████████████████| 55.7MB 76kB/s 
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30


In [49]:
def desSIFT(image):
    sift = cv2.SIFT_create()
    kp, des = sift.detectAndCompute(image,None)
    #draw keypoints
    #import matplotlib.pyplot as plt		
    #img2 = cv2.drawKeypoints(img,kp,None,(255,0,0),4)
    #plt.imshow(img2),plt.show()
    return kp,des

def describeSURF( image):
    surf = cv2.SURF_create()
    # it is better to have this value between 300 and 500
#     surf.setHessianThreshold(400)
    kp, des = surf.detectAndCompute(image,None)
    return kp,des


def getDescriptors(images, labels_g) : 
    descriptors = []
    labels = []
    count = 0
    
    print (images.shape)
    for image in images : 
        # print (image.shape)
        print(count)
        #Converting the image into grayscale         
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
        #Re-sizing the image
        # image = cv2.resize(image, (150, 150), interpolation=cv2.INTER_AREA)
        kp, des = desSIFT(image)
        
        if des is not None : 
            print (des.shape)
            descriptors.append(des)
            labels.append(int(labels_g[count]))
        count += 1
            
    
    print (len(labels))
    print(descriptors[0])
    descriptorsFin = descriptors[0]
    print(len(descriptors[1]))

    # for descriptor in descriptors[1:]:
    #     if descriptor is not None:
    #         descriptorsFin = np.vstack((descriptorsFin, descriptor))
    descriptorsFin = np.asarray(np.concatenate(descriptors, axis = 0))
        
    return descriptorsFin, descriptors, labels

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [ ]:
# x_train = x_train[1:5000]
# y_test = y_train[1:5000]
# y_test = y_test[1:5000]
# y_test = y_test[1:5000]

from sklearn.model_selection import train_test_split

# creating matrix
# input_matrix = np.arange(46928*28*28).reshape((46928,28,28))
second_size = 15000/50000

X1, X2 = train_test_split(x_train, test_size=second_size)
Xt1, Xt2 = train_test_split(x_test, test_size=second_size)
Y1, Y2 = train_test_split(y_train, test_size=second_size)
Yt1, Yt2 = train_test_split(y_test, test_size=second_size)

x_train = X2
x_test = Xt2

y_train = Y2
y_test = Yt2
print('X1 shape: ', x_train.shape)
print('X2 shape: ', x_test.shape)
print('X2 shape: ', y_train.shape)
print('X2 shape: ', y_test.shape)

X1 shape:  (15000, 32, 32, 3)
X2 shape:  (3000, 32, 32, 3)
X2 shape:  (15000, 1)
X2 shape:  (3000, 1)


In [50]:
sift_des, descriptors, sift_labels = getDescriptors(np.vstack((x_train, x_test)), np.vstack((y_train, y_test)))

k = 150

ValueError: ignored

In [ ]:
sift_des.shape

(78118, 128)

In [ ]:
len(descriptors)

5985

In [ ]:
voc,  variance = kmeans((sift_des), k, 1)

(18000, 1)


In [ ]:
print (len(sift_labels))
print (len(sift_des))

# Constructing a histogram of k clusters and number of images having those clusters
imageFeatures = np.zeros((len(sift_labels), k), "float32")
for i in range(len(sift_labels)):
    words, distance = vq(descriptors[i],voc)
    for w in words:
        imageFeatures[i][w] += 1

17970
233804


In [ ]:
stdSlr = StandardScaler().fit(imageFeatures)
imageFeatures = stdSlr.transform(imageFeatures)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(imageFeatures, sift_labels, test_size=0.1667, random_state=4)

In [26]:

# Training the train data using knn
clf = cv2.ml.KNearest_create()
clf.train(X_train, cv2.ml.ROW_SAMPLE, np.asarray(y_train, dtype=np.float32))

True

In [27]:

# Predicting the test values
ret, results, neighbours ,dist = clf.findNearest(X_test, k=10)

In [28]:
pred_label = []
for var in results:
    label = var
    pred_label.append(int(label))

print (y_test)
print (pred_label)
    
# Measuring the accuracies
metrics.accuracy_score(y_test, pred_label)

[3, 3, 7, 7, 5, 8, 0, 4, 6, 2, 9, 5, 5, 9, 5, 6, 5, 9, 7, 3, 7, 4, 9, 2, 0, 7, 0, 1, 9, 8, 5, 1, 3, 6, 0, 6, 1, 2, 8, 0, 0, 0, 6, 2, 6, 4, 6, 1, 2, 7, 1, 9, 8, 7, 1, 1, 7, 2, 8, 6, 8, 1, 2, 8, 9, 9, 3, 7, 3, 5, 0, 6, 7, 1, 2, 6, 2, 3, 0, 8, 3, 7, 8, 4, 5, 9, 3, 7, 0, 4, 7, 3, 0, 9, 4, 3, 9, 5, 8, 1, 3, 7, 4, 2, 5, 7, 0, 3, 7, 7, 4, 1, 7, 3, 0, 5, 2, 4, 7, 7, 1, 4, 6, 9, 7, 8, 8, 3, 5, 8, 0, 7, 2, 1, 6, 2, 4, 4, 2, 2, 8, 5, 1, 9, 7, 1, 0, 5, 4, 5, 6, 7, 2, 0, 1, 9, 4, 1, 6, 8, 9, 7, 3, 7, 8, 4, 8, 2, 4, 8, 5, 6, 6, 3, 8, 7, 1, 0, 5, 1, 4, 8, 2, 0, 4, 7, 2, 0, 1, 3, 9, 2, 6, 2, 2, 7, 9, 8, 8, 6, 6, 9, 7, 0, 2, 2, 8, 0, 5, 2, 8, 3, 2, 5, 8, 9, 6, 0, 8, 4, 8, 0, 0, 8, 0, 9, 0, 5, 3, 2, 8, 3, 6, 1, 4, 2, 8, 5, 8, 7, 8, 7, 5, 1, 5, 6, 3, 9, 1, 8, 7, 6, 8, 4, 1, 3, 8, 5, 0, 7, 4, 0, 7, 5, 4, 6, 7, 6, 7, 8, 5, 3, 5, 4, 6, 7, 2, 0, 9, 6, 4, 5, 0, 7, 1, 3, 3, 2, 2, 2, 0, 9, 6, 7, 1, 6, 4, 6, 0, 7, 3, 4, 8, 7, 0, 3, 5, 3, 4, 5, 3, 6, 7, 0, 0, 6, 6, 6, 7, 2, 0, 2, 4, 1, 0, 3, 1, 6, 9, 4, 9, 2, 6, 

0.10113484646194927

In [46]:
# c = 0
# for i in range(len(y_test)):
#   if y_test[i] == pred_label[i]:
#     c+=1
# print(len(y_test))
# c

print(len(sift_labels))
print(len(voc[0]))
descriptors[0]

17970
128


array([[23.,  3.,  3., ...,  0.,  0.,  0.],
       [ 6.,  0.,  1., ...,  0.,  0.,  0.],
       [ 4.,  8., 14., ...,  3.,  1.,  3.],
       ...,
       [ 0.,  0.,  4., ..., 61., 18.,  7.],
       [15.,  0.,  0., ...,  6.,  3., 10.],
       [ 0.,  0.,  1., ...,  0.,  0.,  4.]], dtype=float32)